# Classify avian sequences into domestic and wild

Update on June 30, 2022: I am going to make sure I am not missing any wild bird sequences and am also going to update to the newest data. So I re-downloaded all metadata for every sequence that had an HA gene from GISAID. I also updated thte H5N1 nextstrain build with up to date IRD data and will use that input file. 

August 18, 2020

To start on this avian flu project, I would like to use the H5N1 sequences that I have already curated for the avian-flu nextstrain build. While some of the sequences on that site are derived from the Influenza Research Database (which pulls from Genbank), others come from gisaid. Gisaid has an annotation feature where avian sequences can be annotated as domestic or wild, which is really helpful. However, most sequences are not annotated. I figured out when writing the application that sequences can generally be classified by reading the associated abstract or paper. To do that, I need to match gisaid ids with genbank ids, and use those to pull abstracts. In this notebook, I will do the following: 

1. I downloaded all available H5N1 sequences from gisaid. I did not stipulate whether they required any given segment or any minimum segment length. This downloaded metadata file should provide all the available metadata for all gisaid h5n1 segments. 
2. Read in this gisaid metadata file to a dictionary containing for each accession number, all of the associated metadata. 
3. Next, I will read in all of the fasta files for H5N1 that are currently used for the avian-flu repo. These fasta files live in `avian-flu/data/`. I have already done a bit of work to classify hosts for the avian flu build as bird, human, nonhuman mammal, and environment. I feel pretty confident that the majority of sequences that are ambiguous will be goose and duck. So, for any sequence annotated as bird whose host in the strain name is goose or duck, then I will pull out the accession number. If the accession is a gisaid accession, then I will check the metadata dictionary to determine whether a. there is a domestic/wild annotation and b. whether there is an associated genbank id.
4. If there is no annotation and no genbank id, then it will be really challenging to classify. If there is an annotation, then we will use that. If there is no annotation, but there is a genbank id, then I will record that genbank id and use it to look up and output an associated abstract for that sequence. 

In [72]:
# using the Entrez feature from bio
from Bio import Entrez
from Bio import SeqIO
import re
import time, datetime
import pandas as pd
import os

## Read in new and old metadata files 

In [73]:
def read_existing_metadata_file(old_metadata_file):
    
    output_dict = {}
    
    with open(old_metadata_file, "r") as infile: 
        for line in infile: 
            if line.startswith("strain") != True:
                strain = line.split("\t")[0]
                virus = line.split("\t")[1]
                isolate_id = line.split("\t")[2]
                date = line.split("\t")[3]
                region = line.split("\t")[4]
                country = line.split("\t")[5]
                division = line.split("\t")[6]
                location = line.split("\t")[7]
                host = line.split("\t")[8]
                originating_lab = line.split("\t")[9]
                submitting_lab = line.split("\t")[10]
                host_species = line.split("\t")[11]
                host_species_standardized = line.split("\t")[12]
                domestic_wild = line.split("\t")[13]
                annotation_method = line.split("\t")[14]
                genbank_id = line.split("\t")[15]
                pubmed_id = line.split("\t")[16].strip()
                
                output_dict[strain] = {"virus":virus, "isolate_id":isolate_id, "date":date, "region":region, 
                                      "country":country, "division":division, "location":location, "host":host, 
                                      "originating_lab":originating_lab, "submitting_lab":submitting_lab, 
                                      "host_species":host_species, "host_species_standardized":host_species_standardized, 
                                      "domestic_wild":domestic_wild, "annotation_method":annotation_method, 
                                       "genbank_id":genbank_id, "pubmed_id":pubmed_id}
    return(output_dict)

In [195]:
def read_new_metadata_file(new_metadata_file, old_metadata_dict, new_strains_file):
    
    with open(new_strains_file, "w") as outfile: 
        outfile.write("")
    
    with open(new_metadata_file, "r") as infile: 
        for line in infile: 
            if line.startswith("strain") == True: 
                with open(new_strains_file, "a") as outfile: 
                    outfile.write(line)
            else: 
                strain = line.split("\t")[0]
                
                if strain not in old_metadata_dict: 
                    with open(new_strains_file, "a") as outfile: 
                        outfile.write(line)

## Read in new gisaid excel file, clean it up, and read in as metadata

In [262]:
"""remove non-ascii characters that end up in these gisaid columns. the \\xd{0D} shows up in text wrangler as 
an upside down question mark"""

def clean(string: str = '') -> str:
    if type(string) == str:
        
        #return(re.sub(r'[^\x00-\x7F]+',' ', string)
        return string.replace('\r\n', '').replace('\n', '').replace('&quot;', '"').replace('\\x{0D}', '')

In [263]:
def clean_excel_file(input_excel_metadata):
    output_filename = input_excel_metadata.replace(".xls","-cleaned.tsv")
    
    """in order to remove stupid non-ascii characters, the only way is to remove them while reading in with the
    converters argument. This allows you to specify how to clean certain columns with custom functions. I want to 
    run clean, which removes non-ascii characters, on all columns. So this first bit is to read in the column 
    names and generate the converters dictionary""" 
    d = pd.read_excel(input_excel_metadata, header=0)
    converters = {x: clean for x in d.columns}
    
    """Then, read in again, do 1 more find and replace (this is necessary still somehow) and write to csv"""
    df = pd.read_excel(input_excel_metadata, header=0, converters = converters)
    
    # replace newlines in columns 
    df = df.replace('\n','', regex=True)    
    
    # write to tsv
    df.to_csv(output_filename, sep="\t", index=False)
    return(output_filename)

In [220]:
def read_in_metadata_file(file):
    count = 0
    metadata_dict = {}
    genes = {"PB2":{"gisaid_id":1,"genbank_id":53},
            "PB1":{"gisaid_id":2,"genbank_id":54},
            "PA":{"gisaid_id":3,"genbank_id":55},
            "HA":{"gisaid_id":4,"genbank_id":56},
            "NP":{"gisaid_id":5,"genbank_id":57},
            "NA":{"gisaid_id":6,"genbank_id":58},
            "MP":{"gisaid_id":7,"genbank_id":59},
            "NS":{"gisaid_id":8,"genbank_id":60},}
    
    with open(file, "r") as infile:
        for line in infile:
            count += 1
            if len(line.split("\t")) < 63:
                print(count)
            strain_name = line.split("\t")[11].strip()
            passage_history = line.split("\t")[14].strip()
            host = line.split("\t")[16].strip()
            vacc_status = line.split("\t")[47].strip()
            domestic_status = line.split("\t")[51].strip()

            for gene in genes: 
                gisaid_id_column = genes[gene]['gisaid_id']
                gisaid_id = line.split("\t")[gisaid_id_column].split("|")[0]
                genbank_id_column = genes[gene]['genbank_id']
                genbank_id = line.split("\t")[genbank_id_column]

                # add to dictionary 
                metadata_dict[gisaid_id] = {"genbank_id":genbank_id,"strain":strain_name,"passage":passage_history,
                                           "vaccination_status":vacc_status,"domestic_wild":domestic_status,"host":host}

        return(metadata_dict)

## Read in files 

This includes: 
1. avian species synonsyms: to standardize host species 
2. exclude file: to exclude specfic strains 
3. strain host fixes: this includes strains for which I just manually annotated the host because it was obvious to a human, but would have been not obvious programmatically 
4. classified abstracts: a text file containing a list of abstracts and their classifications 
5. individual strains from abstracts: for papers where there was a mix of strains domestic and wild, I sometimes annotated them one by one 

In [137]:
"""I have an avian species synonyms document which both corrects spelling errors and standardizes avian host
annotations so that they all match. This document also contains assignments for these species as to whether they
are domestic or wild. Some species like duck and goose are not specific enough, but others like heron are. This
function will read in that file and output a dictionary with the original species name, the corrected name, and the 
domestic/wild classification"""

def read_in_avian_species_synonyms(synonyms_doc):
    fixes_dict = {}
    with open(synonyms_doc, "r") as infile: 
        for line in infile: 
            if not line.startswith("#"):
                if len(line.split("\t")) == 3:
                    species = line.split("\t")[0]
                    fix = line.split("\t")[1]
                    domestic_wild = line.split("\t")[2].strip()

                    fixes_dict[species] = {"fix":fix, "domestic_wild":domestic_wild}
                else:
                    species = line.split("\t")[0].strip()
                    fixes_dict[species] = {"fix":"", "domestic_wild":""}
    return(fixes_dict)

In [138]:
"""some strains have weird passage histories or should otherwise be excluded. This is a list to hold those"""

def read_in_exclude_file(exclude_file):
    exclude_dict = {}
    
    with open(exclude_file, "r") as infile: 
        for line in infile: 
            if "#" not in line and line != "\n":
                strain_name = line.split("\t")[0]
                accession = line.split("\t")[1]
                exclude_dict[strain_name] = accession
                
    return(exclude_dict)

In [139]:
"""some strains are improperly formatted and need to have a host annotation added explicitly"""

def read_in_strain_host_fixes(strain_host_fixes_file):
    
    strain_host_fixes_dict = {}
    
    with open(strain_host_fixes_file, "r") as infile: 
        for line in infile: 
            if "#" not in line:
                strain_name = line.split("\t")[0]
                host_annotation = line.split("\t")[1]
                corrected_strain = line.split("\t")[2]
                strain_host_fixes_dict[strain_name] = {"host_species":host_annotation,"corrected_strain":corrected_strain}
                
    return(strain_host_fixes_dict)

In [140]:
def read_in_classified_abstracts(abstracts_classified_file):
    output_dict = {}
    
    with open(abstracts_classified_file, "r") as infile:
        for line in infile:
            if line != "\n":
                PMID = line.split("\t")[0]
                domestic_wild = line.split("\t")[1]
                notes = line.split("\t")[2]

                output_dict[PMID] = domestic_wild
    return(output_dict)

In [136]:
def read_in_strain_specific_classifications_file(strain_classification_file):
    
    output_dict = {}
    with open(strain_classification_file, "r") as infile: 
        for line in infile:
            if line != "\n":
                strain_name = line.split("\t")[0]                
                domestic_wild = line.split("\t")[1]
                PMID = line.split("\t")[2].strip()
                
                if PMID not in output_dict: 
                    output_dict[PMID] = {}
                
                output_dict[PMID][strain_name] = domestic_wild
    return(output_dict)

## Standardize host names and such 

In [141]:
"""given the species fixes in the annotations file, standardize the species annotation"""

def standardize_host_species(strain_name, host_group, fixes_dict):
    
    # standardize host species names using fix file and and categorize domestic/wild by species if possible
    if host_group.lower() == "avian": 
        host_species = strain_name.split("/")[1].lower()
            
        if host_species in fixes_dict:
            species_fix = fixes_dict[host_species]['fix']
            
        else:
            print("not in dictionary: ",host_species, strain_name)
            species_fix = host_species
                
    else:
        host_species = host_group
        species_fix = host_group

    return(host_species, species_fix)

In [142]:
"""given a host species, determine whether that species has a clear domestic/wild classification"""

def check_for_annotation_by_species(host_species,fixes_dict):
    
    if host_species in fixes_dict:
        domestic_wild_status = fixes_dict[host_species]['domestic_wild']
    
    else:
        domestic_wild_status = ""
    
    return(domestic_wild_status)

In [143]:
"""given an accession, check to see whether it is from gisaid and if so, whether it has a domestic wild annotation"""

def check_for_annotation_by_gisaid(accession,metadata_dict):
    
    if accession.startswith("EPI") and len(accession) == 10 and accession in metadata_dict:
        domestic_wild_status = metadata_dict[accession]['domestic_wild'].lower()
    else:
        domestic_wild_status = ""
        
    return(domestic_wild_status)

In [144]:
def return_status_and_annotation_method(domestic_wild_status_by_species,domestic_wild_status_by_gisaid):
    
    # first check to make sure that the different methods did not return conflicting annotations
    if domestic_wild_status_by_species != "" and domestic_wild_status_by_gisaid != "" and domestic_wild_status_by_species != domestic_wild_status_by_gisaid:
        print("conflicting annotations ",domestic_wild_status_by_species,domestic_wild_status_by_gisaid)
    
    # if the species is informative and gisaid had an annotation
    elif domestic_wild_status_by_species != "" and domestic_wild_status_by_gisaid != "" and domestic_wild_status_by_species == domestic_wild_status_by_gisaid:
        domestic_wild = domestic_wild_status_by_species
        annotation_method = "species name and gisaid"
    
    # if only the species could be used the classify
    elif domestic_wild_status_by_species != "" and domestic_wild_status_by_gisaid == "":
        annotation_method = "species name"
        domestic_wild = domestic_wild_status_by_species
    
    # if only gisaid could be used to classify
    elif domestic_wild_status_by_species == "" and domestic_wild_status_by_gisaid != "":
        annotation_method = "gisaid annotation"
        domestic_wild = domestic_wild_status_by_gisaid
        
    # if neither could be used to classify
    elif domestic_wild_status_by_species == "" and domestic_wild_status_by_gisaid == "":
        domestic_wild = ""
        annotation_method = ""
    
    # if there is somehow another option I'm not thinking of
    else:
        print("something weird is happening!", domestic_wild_status_by_species,domestic_wild_status_by_gisaid)
        
    return(domestic_wild, annotation_method)

In [145]:
"""given an accession number, determine if it is from gisaid or genbank. If from gisaid, look up the corresponding
genbank id for it if available. If not, return a blank string"""

def return_genbank_accession(accession,metadata_dict):
    
    accession_stripped = accession.replace("_","").replace("-","")
 
    # if the accession is a gisaid accession
    if accession_stripped.startswith("EPI"):
        if accession_stripped in metadata_dict:
            genbank_id = metadata_dict[accession_stripped]['genbank_id']
        else: 
            genbank_id = ""
            
    elif len(accession_stripped) == 8: 
        genbank_id = accession   #for a genbank lookup, we need to retain the _ and -, but for gisaid, we do not
    
    else:
        print("this does not follow the rules ", accession)
        
    return(genbank_id)

In [184]:
"""given an accession number, pull the genbank entry that it corresponds to"""

def fetch_genbank_entry(accession_number):
    
    if accession_number != "" and accession_number != ", ":
        Entrez.email = "lhmoncla@gmail.com"      # need to enter an email

        # fetch from the nucleotide database, return a genbank object in text format, with id = genbank accession
        handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="text", id=accession_number)
        genbank_result = handle.read()
        
    else: 
        genbank_result = ""
        
    return(genbank_result)

In [147]:
"""given a genbank result, pull the pubmed id from it"""

def return_PMID(genbank_result):
    if genbank_result != "":
        SearchStr = 'PUBMED\\ \\ \\ [0-9]{8}'
        result = re.search(SearchStr,genbank_result)

        if result:
            PMID = result.group(0).split("   ")[1]
        else:
            PMID = ""
            
    else:
        PMID = ""
    
    return(PMID)

In [186]:
def query_genbank(genbank_id):
    from urllib.error import HTTPError

    flag = True
    
    while flag:
        try:
            time.sleep(0.35) # Sleep; genbank will throw an error if you are hitting it with more than 3 hits per second
            genbank_entry = fetch_genbank_entry(genbank_id)
            pubmed_id = return_PMID(genbank_entry)
            flag = False
        except HTTPError as e:
            time.sleep(1)
            print(e)
    
    return(pubmed_id)

In [209]:
def fix_host_species_and_classify(metadata_file, metadata_dict,fixes_dict,exclude_dict,strain_host_fixes_dict,output_file,master_dict):
    
    with open(output_file, "w") as outfile:
        line_to_write = "\t".join(["strain","virus","isolate_id","date","region","country","division","location","host","subtype","originating_lab","submitting_lab","h5_clade","h5_label_clade","host_species","host_species_standardized","domestic_wild","annotation_method","genbank_id","pubmed_id"])
        outfile.write(line_to_write + "\n")
    output_dict = {}
    
    with open(metadata_file, "r") as infile: 
        for line in infile:
            if "originating_lab" not in line:
                strain_name = line.split("\t")[0]
                host_group =  line.split("\t")[8].lower()
                accession = line.split("\t")[2]

                """filter out strains that should be excluded"""
                if strain_name not in exclude_dict:
                    
                    """first, see if we have already classified this strain"""
                    if strain_name in master_dict: 
                        host_species = master_dict[strain_name]['host_species']
                        host_species_standardized = master_dict[strain_name]['standardized_host_species']
                        domestic_wild = master_dict[strain_name]['domestic_wild']
                        annotation_method = master_dict[strain_name]['annotation_method']
                        host_group = master_dict[strain_name]['host_group']
                        pubmed_id = master_dict[strain_name]['pubmed_id']
                        
                        # for these, we still need to look up a new genbank id
                        genbank_id = return_genbank_accession(accession,metadata_dict)
                        
                    else:                 
                        """if the strain is in the strain host fixes doc, add in the host species here; 
                        else, standardize the host species name with the species synonyms"""
                        if strain_name in strain_host_fixes_dict:
                            host_species = strain_host_fixes_dict[strain_name]['host_species']
                            host_species_standardized = strain_host_fixes_dict[strain_name]['host_species']

                        else:
                            host_species,host_species_standardized = standardize_host_species(strain_name, host_group, fixes_dict)


                        """now, if the host group is avian, check for domestic wild status by its species name and by gisaid
                        annotation"""
                        if host_group == "avian":
                            domestic_wild_status_by_species = check_for_annotation_by_species(host_species,fixes_dict)
                            domestic_wild_status_by_gisaid = check_for_annotation_by_gisaid(accession,metadata_dict)
                            domestic_wild,annotation_method = return_status_and_annotation_method(domestic_wild_status_by_species,domestic_wild_status_by_gisaid)                

                        else:
                            domestic_wild = host_group
                            annotation_method = ""

                        """now, return the genbank ids"""
                        genbank_id = return_genbank_accession(accession,metadata_dict)


                        """pull the genbank entry and pubmed id for the associated paper"""
                        pubmed_id = query_genbank(genbank_id)

                    
                    
                    """now write out a dictionary"""
                    output_dict[strain_name] = {"host_species":host_species,"standardized_host_species":host_species_standardized,
                                               "domestic_wild":domestic_wild,"annotation_method":annotation_method,
                                                "genbank_id":genbank_id, "host_group":host_group,"pubmed_id":pubmed_id}

                    line_to_write = "\t".join([host_species,host_species_standardized,domestic_wild,annotation_method,genbank_id,pubmed_id])
                    with open(output_file, "a") as outfile: 
                        outfile.write(line.strip() + "\t" + line_to_write + "\n")
            
    return(output_dict)

## Run on new files

In [151]:
avian_synonyms_file = "/Users/lmoncla/src/h5n1-host-classification/all-avian-species.txt"
strain_host_fixes = "/Users/lmoncla/src/h5n1-host-classification/strain-host-fixes.tsv"
exclude_file = "/Users/lmoncla/src/h5n1-host-classification/exclude.txt"

todays_date = (datetime.datetime.now()).strftime('%Y-%m-%d')
output_file_directory = "/Users/lmoncla/src/h5n1-host-classification/metadata-with-annotations/"

avian_fixes_dict = read_in_avian_species_synonyms(avian_synonyms_file)
to_exclude_dict = read_in_exclude_file(exclude_file)
strain_host_fixes_dict = read_in_strain_host_fixes(strain_host_fixes)

In [152]:
# pull in abstracts and strains classified by abstracts 
abstracts_classified_file = "/Users/lmoncla/src/h5n1-host-classification/abstracts/abstracts-classified-2020-09-28.txt"
strain_classification_file = "/Users/lmoncla/src/h5n1-host-classification/abstracts/strain_specific_annotations.txt"

classified_by_abstract_dict = read_in_classified_abstracts(abstracts_classified_file)
strains_abstract_dict = read_in_strain_specific_classifications_file(strain_classification_file)

### Read in old metadata and make a new folder with a new file containing only the new strains 

In [153]:
old_metadata_file = "../metadata-with-annotations/metadata_h5n1_ha_final_2020-10-02.txt"
old_metadata_dict = read_existing_metadata_file(old_metadata_file)

In [154]:
# make a new directory for new strains being updated today 
todays_date = (datetime.datetime.now()).strftime('%Y-%m-%d')
os.mkdir("../metadata-with-annotations/"+str(todays_date)+"-update")\

In [155]:
# read in new metadata file and output new strains 
new_metadata_file = "/Users/lmoncla/src/avian-flu/results/metadata-with-clade_h5n1_ha.tsv"
new_strains_file = "../metadata-with-annotations/"+str(todays_date)+"-update/new-strains-"+str(todays_date)+".tsv"
read_new_metadata_file(new_metadata_file, old_metadata_dict, new_strains_file)

### read in new gisaid data so that we have all the metadata

In [159]:
# read in new gisaid data into a metadata dictionary 
#metadata_file = "/Users/lmoncla/src/h5n1-host-classification/gisaid-data/gisaid-all-h5n1-2020-08-18.txt"
metadata_file = "/Users/lmoncla/src/h5n1-host-classification/gisaid-data/gisaid-all-h5n1-2022-06-30.xls"

cleaned_metadata_tsv = clean_excel_file(metadata_file)

In [196]:
metadata_dict = read_in_metadata_file(cleaned_metadata_tsv)
#metadata_dict

### Classify the new sequences 

In [210]:
# run for all genes to make sure I have all the hosts I need
all_genes_dict = {}

output_file = new_strains_file.replace(".tsv","-with-annotations.tsv")
gene_dict = fix_host_species_and_classify(new_strains_file, metadata_dict,avian_fixes_dict,to_exclude_dict,strain_host_fixes_dict,output_file,all_genes_dict)
    
# update the dictionary with new entries. This will retain the old keys and values, but add in new ones
all_genes_dict.update(gene_dict)

HTTP Error 500: Internal Server Error


In [211]:
# number of unique, new strains
len(all_genes_dict)

1253

## Pull abstracts to read

Given this new metadata file, for all bird sequences that don't have an annotation yet and do have a pubmed id, pull the abstract so that I can read it to classify. 

In [197]:
def collate_unique_PMIDs(metadata_file, PMIDs, cannot_classify, classify_by_abstract):
    
    with open(metadata_file, "r") as infile: 
        for line in infile: 
            if "originating_lab" not in line:
                strain = line.split("\t")[0]
                host_group = line.split("\t")[8].lower()
                domestic_wild = line.split("\t")[16]
                PMID = line.split("\t")[19].strip()
                
                if host_group == "avian" and domestic_wild == "" and PMID != "":
                    PMIDs.append(PMID)
                    classify_by_abstract.append(strain)
                elif host_group == "avian" and domestic_wild == "" and PMID == "":
                    cannot_classify.append(strain)
    
    PMIDs = list(set(PMIDs))
    cannot_classify = list(set(cannot_classify))
    classify_by_abstract = list(set(classify_by_abstract))
    return(PMIDs, cannot_classify, classify_by_abstract)

In [198]:
def pull_abstract(PMID):
    
    pmid_handle = Entrez.efetch(db='pubmed', id=PMID, retmode='text', rettype='abstract')
    abstract = pmid_handle.read()
    
    return(abstract)

In [199]:
def return_necessary_abstracts(PMIDs, abstracts_output_filename):
    
    with open(abstracts_output_filename, "w") as outfile: 
        outfile.write("")
    
    for p in PMIDs: 
        abstract = pull_abstract(p)
                    
        with open(abstracts_output_filename, "a") as outfile: 
            outfile.write(p + "\n")
            outfile.write(abstract + "\n")

In [202]:
metadata_directory = "/Users/lmoncla/src/h5n1-host-classification/metadata-with-annotations/2022-07-12-update/"
abstracts_output_filename = "/Users/lmoncla/src/h5n1-host-classification/metadata-with-annotations/2022-07-12-update/abstracts-"+todays_date+".txt"

unique_PMIDs = []
cannot_classify = []
classify_by_abstract = []

metadata_input_file = "/Users/lmoncla/src/h5n1-host-classification/metadata-with-annotations/2022-07-12-update/new-strains-2022-07-12-with-annotations.tsv"
unique_PMIDs,cannot_classify,classify_by_abstract = collate_unique_PMIDs(metadata_input_file, unique_PMIDs, cannot_classify, classify_by_abstract)

# print the number of abstracts I need to read, and the number of strains that can't be classified at all, and the 
# number of strains that will be captured by abstract classification
print(len(unique_PMIDs), len(cannot_classify), len(classify_by_abstract))

1 427 1


In [203]:
return_necessary_abstracts(unique_PMIDs, abstracts_output_filename)

## Read in annotations from abstract reading and make the final cut

In [204]:
def read_in_classified_abstracts(abstracts_classified_file):
    output_dict = {}
    
    with open(abstracts_classified_file, "r") as infile:
        for line in infile:
            if line != "\n":
                PMID = line.split("\t")[0]
                domestic_wild = line.split("\t")[1]
                notes = line.split("\t")[2]

                output_dict[PMID] = domestic_wild
    return(output_dict)

In [205]:
def read_in_strain_specific_classifications_file(strain_classification_file):
    
    output_dict = {}
    with open(strain_classification_file, "r") as infile: 
        for line in infile:
            if line != "\n":
                strain_name = line.split("\t")[0]                
                domestic_wild = line.split("\t")[1]
                PMID = line.split("\t")[2].strip()
                
                if PMID not in output_dict: 
                    output_dict[PMID] = {}
                
                output_dict[PMID][strain_name] = domestic_wild
    return(output_dict)

In [212]:
def add_manual_annotations(metadata_file, abstracts_dict, strain_abstract_dict, output_filename):
    
    with open(output_filename, "w") as outfile: 
        outfile.write("")
    
    with open(metadata_file, "r") as infile: 
        for line in infile: 
            if "originating_lab" not in line:
                strain = line.split("\t")[0]
                host_group = line.split("\t")[8].lower()
                current_domestic_wild = line.split("\t")[16]
                current_annotation_method = line.split("\t")[17]
                PMID = line.split("\t")[19].strip()
                
                """add in annotations that I manually curated"""
                if host_group == "avian" and current_domestic_wild == "" and PMID != "":
                    if PMID in abstracts_dict and abstracts_dict[PMID] != "":
                        domestic_wild = abstracts_dict[PMID]
                        annotation_method = "manual curation"
                        
                        
                        """if I needed to annotate each strain separately, look that up in the strains file"""
                        if domestic_wild == "see strain list":
                            if PMID in strain_abstract_dict and strain in strain_abstract_dict[PMID]:
                                domestic_wild = strain_abstract_dict[PMID][strain]
                                annotation_method = "manual curation"
                            else:
                                domestic_wild = ""
                    
                    else:
                        domestic_wild = ""
                        annotation_method = ""
                    
                    first_part = "\t".join(line.split("\t")[0:16])
                    last_part = "\t".join(line.split("\t")[18:])
                    line_to_write = "\t".join([first_part,domestic_wild, annotation_method,last_part])
                
                
                # if it already has an annotation, or if it doesn't but there isn't an abstract, move on
                else:
                    line_to_write = line
                
            else:
                line_to_write = line
            
            """write it out"""
            with open(output_filename, "a") as outfile: 
                outfile.write(line_to_write)

In [22]:
abstracts_classified_file = "/Users/lmoncla/src/h5n1-host-classification/abstracts/abstracts-classified-2020-09-28.txt"
strain_classification_file = "/Users/lmoncla/src/h5n1-host-classification/abstracts/strain_specific_annotations.txt"

In [23]:
classified_by_abstract_dict = read_in_classified_abstracts(abstracts_classified_file)
strains_abstract_dict = read_in_strain_specific_classifications_file(strain_classification_file)

In [24]:
metadata_directory = "/Users/lmoncla/src/h5n1-host-classification/metadata-with-annotations/"
todays_date = (datetime.datetime.now()).strftime('%Y-%m-%d')
genes_to_run = ['pb2','pb1','pa','ha','np','na','mp','ns']

for gene in genes_to_run: 
    metadata_file = metadata_directory + "metadata_h5n1_" + gene + "_with_accessions_2020-10-02.txt"
    output_filename = metadata_directory + "metadata_h5n1_" + gene + "_final_"+ todays_date+".txt"
    add_manual_annotations(metadata_file, classified_by_abstract_dict, strains_abstract_dict, output_filename)